# MegaHacakthon - Voksedigital

### Submitted by: Asif Husain

## Importing Essential Libraries

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
#!pip install imblearn

In [59]:
#!pip install pandas_summary

In [60]:
#!pip install xgboost

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import zscore, uniform, randint

from scipy import stats


from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

#from sklearn.cluster import KMeans
#from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,precision_score,recall_score
#from imblearn.over_sampling import RandomOverSampler

import pickle

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [62]:
#time function to monitor model runtime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [63]:
# Set Options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 600)
%matplotlib inline
warnings.filterwarnings("ignore")

# Import Data

In [64]:
#Function for importing data and checkign their Shape
def Import_data(file_name):
    DF1 = pd.read_csv(file_name)                    #Read CSV file
    return DF1

# Function to remove features with 25%+ Null values

In [65]:
# Function for checking null values and imputing mean for all null values
def treat_null_values(data):
    columns = data.columns
    for col in columns:
        missing_percentage = data[col].isna().sum()/data.shape[0]*100
        col_dtype = data[col].dtype
        #print(col,':',missing_percentage,col_dtype)
        if missing_percentage > 25:
            data = data.drop(col,axis=1)
            print(col,'dropped:',missing_percentage)
        elif col_dtype == 'object' :
            data[col].fillna(data[col].mode()[0],inplace=True)
        else:
            data[col].fillna(data[col].mean(),inplace=True)
    return data

# Unexpected Values

In [66]:
import re

In [67]:
# Function for checking unexpected values in columns
def treat_unexpected_values(data):
    #print(data.columns)
    for col in data.columns[1:]:
        print('\nChecking Column: ',col)
        temp = data[data[col].astype(str).str.contains('[^A-Za-z0-9. ]')]
        if temp.shape[0] == 0:
            print("No Unexpected values found in column",col)
        else:
            print("Unexpected values found in column",col)
            print(temp[col].unique())
                     
            #calculating median values for normal rows
            #median = data[data[col].astype(str).str.contains('[0-9.]')].median()
            
            #imputing median values 
            #data[col].replace('?',str(median[col]),inplace=True)
            
            #print(data[col].mode()[0])
            #data[col].replace('?',str(data[col].mode()[0]),inplace=True)
            
            #print('Imputation Successful')
            
    #return data

# Drop the features which are having same value more than 90%

In [68]:
# Function for checking columns with same value in all rows
def check_low_deviation(data):
    columns = data.columns
    for col in columns[1:-1]:
        #col_dtype = data[col].dtype
        #if col_dtype == 'object':
        #mode = data[col].mode()[0]
        #print(data[col].value_counts())
        temp = data[col].value_counts()
        #print(temp)
        #print(temp.iloc[0])
        mode_percentage = temp.iloc[0]/data.shape[0]*100
        if mode_percentage > 90:
            data = data.drop(col,axis=1)
            print(col,':',mode_percentage)
                
        #sd = data[col].std()
        #if sd == 0:
        #    data = data.drop(col,axis=1)
        #    print(col,':',sd)
    return data

# Duplicates

In [69]:
#Function for checking duplicate rows in data
def treat_duplicates(data):
    duplicate_rows = data[data.duplicated(keep='first')]
    if duplicate_rows.shape[0] == 0:
        print('No duplicate rows in data.')
    else:
        print(duplicate_rows)

# Pie-plots

In [70]:
# Function for creating Pie-chart
def pieplots(data):
    col_names = data.columns[1:]
    for col in col_names:
        print('Pie of column',col)
        temp_data = data[col].value_counts()
        print(temp_data)
        plt.pie(temp_data, labels=temp_data.index, autopct='%.1f%%');
        plt.show()

# Encoding ordinal and nominal colums

In [71]:
""" 
----- Ordinal ---- Custome Encoding
'educ'

----- Nominal ----  One-Hot Encoding
'race', 'father_country','house_stat','major_ind_code','self_country','mother_country','house_summary','hisp_origin','marriage_status','citizenship','fed_liability',
'full_partime_job'

"""

#This function encode categorical data

def encoding(data):
    data= pd.get_dummies(data, columns=['gender','race', 'father_country','house_stat','major_ind_code','mother_country',
                                              'house_summary','hisp_origin','marriage_status','citizenship','fed_liability', 'full_partime_job'],
                               drop_first = True) 
    
    data['educ'].replace([' Associates degree-academic program', ' Children',
       ' Some college but no degree', ' High school graduate',
       ' 10th grade', ' Prof school degree (MD DDS DVM LLB JD)',
       ' Bachelors degree(BA AB BS)', ' 7th and 8th grade',
       ' 5th or 6th grade', ' 9th grade', ' 11th grade',
       ' Masters degree(MA MS MEng MEd MSW MBA)',
       ' Associates degree-occup /vocational',
       ' 1st 2nd 3rd or 4th grade', ' Less than 1st grade',
       ' 12th grade no diploma', ' Doctorate degree(PhD EdD)'], [14, 1, 11, 8, 7, 15, 12, 5, 4, 6, 9, 16, 13, 3, 2, 10, 17],inplace=True)
    
    #data['income'].replace(['More than 50K $', 'Less than 50K $'],[0,1],inplace=True)
    
    return data

In [72]:
def frequency_encoding(data):
    #X = data.drop(['ID','income'],axis=1)
    #Y = data['income']
    #data['income'].replace(['More than 50K $', 'Less than 50K $'],[0,1],inplace=True)
    cat_columns = data.select_dtypes(include=['object']).columns
    for col in cat_columns:
        if col != 'income':
            frequency = data.groupby(col).size()
            #print(frequency)
            #freq_encoded_col = data.groupby([col])['income'].size().to_dict()
            data[col] =  data[col].map(frequency)
    return data

In [73]:
def mixed_encoding(data):
    col_less_categories = []
    cat_columns = data.select_dtypes(include=['object']).columns
    
    # Frequency encoding for columns with higher nos. of categories
    for col in cat_columns:
        if data[col].nunique()>10:
            if col != 'income':
                frequency = data.groupby(col).size()
                data[col] =  data[col].map(frequency)
        else:
            if col != 'income':
                col_less_categories.append(col)
    
    # One-hot encoding for columns with higher nos. of categories
    data= pd.get_dummies(data, columns=col_less_categories,drop_first = True)
            
    return data

# DATA PRE-PROCESSING

In [74]:
# Function to preprocesss the data

def preprocess_data(path_train_csv,path_test_csv,target_feature):
    
    print('\nINITIALISING DATA PRE-PROCESSING')
    
    train = Import_data(file_name = path_train_csv)

    test = Import_data(file_name = path_test_csv)
     
    train1 = treat_null_values(train)
    
    test1 = treat_null_values(test)

    for col_name in ['father_country','mother_country','self_country']:
        train1[col_name].replace(' ?',str(train1[col_name].mode()[0]),inplace=True)
        test1[col_name].replace(' ?',str(test1[col_name].mode()[0]),inplace=True)

    train2 = check_low_deviation(train1)
    test2 = check_low_deviation(test1)

    train_encoded = encoding(train2)
    test_encoded = encoding(test2)
    
    #train_encoded = frequency_encoding(train2)
    #test_encoded = frequency_encoding(test2)
    
    #train_encoded = mixed_encoding(train2)
    #test_encoded = mixed_encoding(test2)
    
    #print(train_encoded.head())
    
    #print(train_encoded.info())
    
    train_encoded[target_feature].replace(['More than 50K $', 'Less than 50K $'],[0,1],inplace=True)
    train_encoded[target_feature] = train_encoded[target_feature].astype('category')
    
    regex = re.compile(r"\[|\]|<| ", re.IGNORECASE)
    train_encoded.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<',' '))) else col for col in train_encoded.columns.values]
    test_encoded.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<',' '))) else col for col in test_encoded.columns.values]
    
    #print(train_encoded.head())
    #print(train_encoded.info())
    
    #print(test_encoded.head())
    #print(test_encoded.info())
    
    #X_Scaled = X_data.apply(zscore)
      
    # Impute duplicate values
    #df.drop_duplicates(inplace=True)
    
    #print(train_encoded.columns)
    #print(test_encoded.columns)
    
    return train_encoded,test_encoded,target_feature

# MODEL BUILDING

In [75]:
# Function to try various models and get performances of all the models
def train_model(preprocessed_df,target_feature):

    print('\nINITIALISING MODEL TRAINING')
    
    # Initialize variables
    model_list = []
    train_acc_list = []
    test_acc_list = []
    test_precision_list = []
    test_recall_list = []
    
    X = preprocessed_df.copy()
    X.drop('ID',axis=1,inplace=True)
    X.drop(target_feature,axis=1,inplace=True)
    y = preprocessed_df[target_feature]
    
    X_Scaled = X.apply(zscore)

    #print(X_Scaled.columns[X_Scaled.columns.str.contains("[\[|\]|<| ]")])
    regex = re.compile(r"\[|\]|<| ", re.IGNORECASE)
    X_Scaled.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<',' '))) else col for col in X_Scaled.columns.values]
    #test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<',' '))) else col for col in test.columns.values]

    # Split into train and test
    X_train,X_test,y_train,y_test = train_test_split(X_Scaled,y,test_size=0.3,random_state=5)
    
    print(X_train.shape)
    print(y_train.shape)
    
    # Intialize models
    lr = LogisticRegression(random_state=5)
    #nb = GaussianNB()
    svcm = svm.SVC(random_state=5)
    dt = DecisionTreeClassifier(random_state=5)
    rf = RandomForestClassifier(random_state=5)
    xgb = XGBClassifier(random_state=5)
    ada = AdaBoostClassifier(random_state=5)
    gdbc = GradientBoostingClassifier(random_state=5)
    knn = KNeighborsClassifier()
    model_list = [lr,dt,rf,xgb,ada,gdbc]
    #model_list = [lr,dt,rf,xgb,ada]
    #model_list = [ada]
    
    col_names = ['Model','Train_accuracy','Test_accuracy','Test_precision','Test_recall']
    model_performance = pd.DataFrame(columns=col_names)
    
    Interate through models, train and measure performance
    for idx,i in enumerate(model_list):
      i.fit(X_train,y_train)
      train_pred= i.predict(X_train)
      test_pred = i.predict(X_test)
      print(i)
                
    '''for idx,i in enumerate(model_list):
        if i == xgb:
            model_name = 'xgb'
        elif i == lr:
            model_name = 'lr'
        elif i == dt:
            model_name = 'dt'
        elif i == gdbc:
            model_name = 'gdbc'
        elif i == ada:
            model_name = 'ada'
        elif i == rf:
            model_name = 'rf'
        elif i == svcm:
            model_name = 'svcm'
        elif i == knn:
            model_name = 'knn'
        train_pred,test_pred = hyperparameter_tuning(i,model_name,X_train,y_train,X_test)'''
        

        
        train_acc = accuracy_score(y_train,train_pred)
        test_acc = accuracy_score(y_test,test_pred)
        test_precision = precision_score(y_test,test_pred,average='weighted')
        test_recall = recall_score(y_test,test_pred,average='weighted')

        model_performance.loc[len(model_performance.index)] = [i,train_acc,test_acc,test_precision,test_recall]
        
    print(model_performance)
        
    return model_performance,X_train,y_train

SyntaxError: ignored

In [ ]:
def hyperparameter_tuning(model,model_name,X_train,y_train,X_test):
    
    if model_name == 'xgb':
        param = {
            'learning_rate': uniform(.05, .3)
            , 'subsample': uniform(.2, .3)
            , 'n_estimators': randint(20, 70)
            , 'min_child_weight': randint(1, 40)
            , 'reg_alpha': uniform(0, .7)
            , 'gamma': uniform(0, .7)
            , 'reg_lambda': uniform(0, .7)
            , 'colsample_bytree': uniform(.1, .7)
            , 'max_depth': randint(2, 6)
        }
    elif model_name == 'lr':
        param = {
            'penalty': ['l1', 'l2', 'elasticnet']
            , 'C': uniform(0.001,100)
            , 'fit_intercept': [True,False]
            , 'class_weight': ['balanced',None]
            , 'solver': ['newton-cg','lbfgs','sag','saga']
            , 'multi_class': ['auto','over','multinomial']
            , 'l1_ratio': uniform(0.01, 0.99)
            , 'warm_start': [True,False]
        }
    elif model_name == 'dt':
        param = {
            'criterion': ['gini', 'entropy', 'log_loss']
            , 'splitter': ['best','random']
            , 'max_depth': randint(2,30)
            , 'max_features': ['auto','sqrt','log2',None]
            , 'class_weight': ['balanced',None]
        }
    elif model_name == 'gdbc':
        param = {
            'loss': ['log_loss', 'deviance', 'exponential']
            , 'learning_rate': uniform(.0, 100)
            , 'criterion': ['friedman_mse','squared_error','mse']
            #, 'max_depth': randint(2,15)
            , 'max_features': ['auto','sqrt','log2',None]
            #, 'ccp_alpha': uniform(0.01, 100)
        }
    elif model_name == 'ada':
        param = {
            'n_estimators': randint(10,200)
            , 'learning_rate': uniform(.001, 100)
            , 'algorithm': ['SAMME','SAMME.R']
        }
    elif model_name == 'rf':
        param = {
            'n_estimators': randint(10,200)
            , 'criterion': ['gini','entropy','log_loss']
            , 'max_depth': randint(2,30)
            , 'max_features': ['sqrt','log2',None]
            , 'bootstrap': [True,False]
            , 'class_weight': ['balanced','balanced_subsample',None]
            , 'max_samples': uniform(0.0,1.0)
        }
    elif model_name == 'svcm':
        param = {
            'C': uniform(0.001,100)
            , 'kernel': ['linear','rbf','sigmoid','precomputed']
            , 'gamma': uniform(0,100)
            , 'shrinking': ['True','False']
            , 'decision_function_shape': ['ovo','ovr']
            , 'class_weight': ['balanced',None]
        }
    
    
    
    randomsearch = RandomizedSearchCV(model,param_distributions=param,n_iter=50,n_jobs=-1,scoring='accuracy',return_train_score=False,verbose=5,random_state=5)
    
    start_time = timer(None) # timing starts from this point for "start_time" variable
    print('Model:',model_name)
    randomsearch.fit(X_train, y_train)
    timer(start_time) # timing ends here for "start_time" variable
    
    train_pred= randomsearch.predict(X_train)
    test_pred = randomsearch.predict(X_test)
    
    return train_pred, test_pred

In [ ]:
# Function to pickle the model with best weights
def pickle_prediction(model_performance,preprocessed_train,preprocessed_test,target_feature,file_name):
    
    best_model = model_performance[model_performance['Test_accuracy']==max(model_performance['Test_accuracy'])]['Model'].iloc[0]
    
    X_train = preprocessed_train.copy()
    X_train.drop('ID',axis=1,inplace=True)
    X_train.drop(target_feature,axis=1,inplace=True)
    y_train = preprocessed_train[target_feature] 
    X_Scaled = X_train.apply(zscore)
    
    #print(X_Scaled.info())
    #print(preprocessed_test.info())
    
    X_test = preprocessed_test.copy()
    X_test.drop('ID',axis=1,inplace=True)
    X_test_Scaled = X_test.apply(zscore)
    
    print(best_model)
    
    best_model.fit(X_Scaled,y_train)
    with open(file_name, 'wb') as handle:
        pickle.dump(best_model, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    test_pred = best_model.predict(X_test_Scaled)
    
    submission_df = pd.read_csv('/content/drive/Othercomputers/My Laptop/Desktop/PGPAIML/Code/Voksd Hackathon/Sample_Submission.csv')
    submission_df.loc[:,'income'] = test_pred    
    submission_df.to_csv('/content/drive/Othercomputers/My Laptop/Desktop/PGPAIML/Code/Voksd Hackathon/Submission_df.csv',sep=',',index = False)

In [ ]:
# Main function to call the pipeline
def main_fn():
    
    path_train_csv = '/content/drive/Othercomputers/My Laptop/Desktop/PGPAIML/Code/Voksd Hackathon/train.csv'
    path_test_csv = '/content/drive/Othercomputers/My Laptop/Desktop/PGPAIML/Code/Voksd Hackathon/test.csv'
    target_feature = 'income'
    
    preprocessed_train,preprocessed_test,target_feature = preprocess_data(path_train_csv,path_test_csv,target_feature)
    
    model_performance,X_train,y_train = train_model(preprocessed_train,target_feature)
    
    model_performance.to_csv('/content/drive/Othercomputers/My Laptop/Desktop/PGPAIML/Code/Voksd Hackathon/model_performance.csv',sep=',')
    
    pickle_prediction(model_performance,preprocessed_train,preprocessed_test,target_feature,'model.pkl')
    

In [ ]:
main_fn()